In [1]:
import scrapy, re, urllib, datetime, io, requests
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.exporters import CsvItemExporter
from scrapy.linkextractors import LinkExtractor
from scrapy.item import DictItem, Field

from bs4 import BeautifulSoup as bs

In [2]:
from scrapy.item import DictItem, Field

def create_item_class(class_name,field_list):
    field_dict = {}
    for field_name in field_list:
        field_dict[field_name] = Field()

    return type(class_name, (DictItem,), {'fields':field_dict})


In [3]:
class Pipeline(object):
    """Scrapy Pipeline Object for Links, Results, Team, Individual"""
    
    def __init__(self):
        self.files = {}
        self.exporters = {}
    
    def create_file(self, file): 
        # if the file doesn't exist create it 
        if file not in self.files:
            self.files[file] = open(file + '.csv', 'wb')
            self.exporters[file] = CsvItemExporter(self.files[file])
            self.exporters[file].start_exporting()
    
    def close_spider(self, spider):
        [e.finish_exporting() for e in self.exporters.values()]
        [f.close() for f in self.files.values()]
 
    def process_item(self, item, spider):
        # Define what item type it is 
        file = type(item).__name__.replace("item_", "").lower()
        self.create_file(file)
        
        # Export 
        self.exporters[file].export_item(item)
        
        return item

In [4]:
class StatsSpider(scrapy.Spider): 
    name = "Stats"
    regex = '.*\/stats\?id\=\d+\&year_stat_category_id\=\d+'
    xpath = "//body//div[@id='contentarea']//div[@id='stats_div']//table"
    
    def start_requests(self): 
        file_in = 'links.csv'

        # Pull in the data 
        data = pd.read_csv(file_in)
        stats = data[data.key == 'stats'].drop_duplicates()
        stats.sort_values(['team'], inplace = True)
        stats_list = stats.to_dict(orient='record')
        
        # Loop through the links 
        for url in stats_list[:1]: 
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse,
                                 meta = {'team':url['team']}
                                )
    
    def parse(self, response): 
        # Year 
        year = response.selector.xpath("//body//div[@id='contentarea']//fieldset//div/" +
                 "/form[@id='change_sport_form']//select[@id='year_list']//option[@select" +
                 "ed='selected']//text()").extract()[0]
        
        # Get all the links on the page
        le = LinkExtractor(restrict_xpaths = (self.xpath,),
                           allow = self.regex
                          ) 
        
        links = le.extract_links(response)
        
        # Build up the output 
        output = []
        for link in links: 
            yield scrapy.Request(link.url, 
                                 callback=self.parse_stats,
                                 meta = {'year':year, 'team':response.meta['team'], 'stat':link.text}
                                )
    
    def parse_stats(self, response):
        print(response.meta['stat'], response.meta['year'], response.meta['team'])
        # Parse tables 
        table = self.table_cleaner(html = response.body, 
                                   target_table = 2, 
                                   stat = response.meta['stat'], 
                                   year = response.meta['year'], 
                                   team = response.meta['team'],
                                   trim_n_rows = 3
                                  )
        # Create dynamic items
        stat = response.meta['stat'].replace(" ","").replace("/","").replace(".","")
        field_list = table.columns
        DynamicItem = create_item_class(stat, field_list)
        item = DynamicItem()
        
        # yield items
        for record in table.to_dict(orient='record'):
            for k, v in record.items(): 
                item[k] = v
            yield item
    
    def table_cleaner(self, html, target_table, stat, year, team, trim_n_rows = 0):
        """Read HTML string and return a table
           html : HTML string
           target_table : Target table number
           trim_n_rows : Trim X number of rows from the end of the table (enter as positive number)
        """
        # Create tables
        tables = pd.read_html(html) 
        # Subset to single table of interest 
        table = tables[target_table] 
        # NCAA kicks out unlabeled columns for some reason. Find the first instance and use it to slice
        stop_index = [1 if 'Unnamed: ' in column  else 0 for idx, column in enumerate(table.columns)].index(1)
        # Fix the column names so no spaces, /'s, or .'s 
        columnNamesFixed = {col:col.replace(" ","").replace("/","").replace(".","") for col in table.iloc[:,:stop_index].columns}
        table.rename(columns = columnNamesFixed, inplace = True)
        # Clean up table 
        table = table[:-trim_n_rows].iloc[:,:stop_index]
        # Add static variables 
        table['stat'] = stat
        table['year'] = year
        table ['team'] = team
        return table 

In [5]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                          'ITEM_PIPELINES': {'__main__.Pipeline':100}
                         })

process.crawl(StatsSpider)
process.start()

2017-12-10 20:19:13 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-10 20:19:13 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-10 20:19:13 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-10 20:19:13 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

Passing 2014-15 Air Force Falcons


2017-12-10 20:19:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10762>
{'CompletionPct': nan,
 'Completions': nan,
 'CompletionsPerGame': 0.0,
 'G': 13,
 'GP': '13',
 'GS': '13',
 'Interceptions': nan,
 'Jersey': '87',
 'PassAttempts': nan,
 'PassEff': nan,
 'PassTDs': nan,
 'PassYards': nan,
 'PassYardsG': 0.0,
 'Player': 'Hansen, Alexander',
 'Pos': 'DL',
 'YdsPerCompletion': nan,
 'Yr': 'Jr',
 'stat': 'Passing',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10776> (referer: http://stats.ncaa.org/team/721/stats/11980)
2017-12-10 20:19:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10777> (referer: http://stats.ncaa.org/team/721/stats/11980)
2017-12-10 20:19:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET h

Participation 2014-15 Air Force Falcons


2017-12-10 20:19:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10778>
{'G': 13,
 'GP': '13',
 'GS': '13',
 'Jersey': '87',
 'Player': 'Hansen, Alexander',
 'Pos': 'DL',
 'Yr': 'Jr',
 'stat': 'Participation',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10762>
{'CompletionPct': nan,
 'Completions': nan,
 'CompletionsPerGame': 0.0,
 'G': 13,
 'GP': '13',
 'GS': '6',
 'Interceptions': nan,
 'Jersey': '20',
 'PassAttempts': nan,
 'PassEff': nan,
 'PassTDs': nan,
 'PassYards': nan,
 'PassYardsG': 0.0,
 'Player': 'McHenry, Gavin',
 'Pos': 'DB',
 'YdsPerCompletion': nan,
 'Yr': 'Jr',
 'stat': 'Passing',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id

...

2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10769>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Int': nan,
 'IntRYds': nan,
 'IntRetTDs': nan,
 'Jersey': '19',
 'PBU': nan,
 'PDef': 0,
 'Player': 'Hill, Brayden',
 'Pos': 'DB',
 'Yr': 'So',
 'stat': 'Passes Defended',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10769>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Int': nan,
 'IntRYds': nan,
 'IntRetTDs': nan,
 'Jersey': '3W',
 'PBU': nan,
 'PDef': 0,
 'Player': 'Bronkar, Cody',
 'Pos': 'WR',
 'Yr': 'Fr',
 'stat': 'Passes Defended',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10769>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Int': nan,
 'IntRYds': na

2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10765>
{'APY': 15,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '33',
 'KORetYds': nan,
 'Player': 'Clinton-Earl, Aaron',
 'Pos': 'RB',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': 15.0,
 'Yr': 'Jr',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10765>
{'APY': 0,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '19',
 'KORetYds': nan,
 'Player': 'Hill, Brayden',
 'Pos': 'DB',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': nan,
 'Yr': 'So',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrap

2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10766>
{'FG': nan,
 'FumbRetPAT': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '19',
 'KickPAT': nan,
 'KickReturnPAT': nan,
 'PATAtt': nan,
 'Pass2PTAtt': nan,
 'Player': 'Hill, Brayden',
 'Points': 0,
 'Pos': 'DB',
 'ReceivingPAT': nan,
 'Ru2PTAtt': nan,
 'RushPAT': nan,
 'Safeties': nan,
 'TDs': nan,
 'Yr': 'So',
 'stat': 'Scoring',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10769>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Int': nan,
 'IntRYds': nan,
 'IntRetTDs': nan,
 'Jersey': '89',
 'PBU': nan,
 'PDef': 0,
 'Player': 'Zuberer, Grayson',
 'Pos': 'TE',
 'Yr': 'So',
 'stat': 'Passes Defended',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http

2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10765>
{'APY': 0,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '89',
 'KORetYds': nan,
 'Player': 'Zuberer, Grayson',
 'Pos': 'TE',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': nan,
 'Yr': 'So',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11980&year_stat_category_id=10765>
{'APY': 0,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '90',
 'KORetYds': nan,
 'Player': 'Allen, Reed',
 'Pos': 'DL',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': nan,
 'Yr': 'Jr',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2014-15'}
2017-12-10 20:19:25 [scrapy.core.

2017-12-10 20:19:25 [scrapy.core.engine] INFO: Closing spider (finished)
2017-12-10 20:19:25 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 10109,
 'downloader/request_count': 18,
 'downloader/request_method_count/GET': 18,
 'downloader/response_bytes': 170795,
 'downloader/response_count': 18,
 'downloader/response_status_count/200': 18,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 12, 11, 2, 19, 25, 949456),
 'item_scraped_count': 1376,
 'log_count/DEBUG': 1395,
 'log_count/INFO': 7,
 'memusage/max': 123633664,
 'memusage/startup': 123633664,
 'request_depth_max': 1,
 'response_received_count': 18,
 'scheduler/dequeued': 18,
 'scheduler/dequeued/memory': 18,
 'scheduler/enqueued': 18,
 'scheduler/enqueued/memory': 18,
 'start_time': datetime.datetime(2017, 12, 11, 2, 19, 13, 113043)}
2017-12-10 20:19:25 [scrapy.core.engine] INFO: Spider closed (finished)
